<a href="https://colab.research.google.com/github/xkaska02/form-data-extraction/blob/dev/finetune_lilt_on_forms_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

In [ ]:
!pip install -q evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install wandb

In [ ]:
from google.colab import userdata
import wandb
import os

wandb.login(key=userdata.get('WANDB_API_KEY'))
# wandb.init(project="form-data-extraction")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xkaska (xkaska-brno-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from transformers import LiltForTokenClassification, LiltModel
import torch.nn as nn
class LiltForTokenClassification2layer(LiltForTokenClassification):
  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.lilt = LiltModel(config, add_pooling_layer=False)

    self.classifier = nn.Sequential(
        nn.Linear(config.hidden_size, config.hidden_size),
        nn.ReLU(),
        nn.Dropout(config.hidden_dropout_prob),
        # nn.Linear(config.hidden_size, config.hidden_size),
        # nn.ReLU(),
        # nn.Dropout(config.hidden_dropout_prob),
        nn.Linear(config.hidden_size, config.num_labels)
    )

    self.post_init()

In [ ]:
from datasets import load_dataset, DatasetDict
# dataset = load_dataset("json", data_files={"train":"train_split.json", "validation":"validation_split.json", "test":"test_split.json"})
dataset = load_dataset("json", data_files={"train":"train_split.json", "validation":"validation_split75.json", "test":"test_split.json"})

In [ ]:
print(dataset["train"][0])

{'ner_tags': [3, 3, 0, 0, 4, 0, 0, 4, 4, 4, 4, 4, 5, 0, 0, 0, 5, 5, 0, 0, 5, 5, 6, 0, 0, 0, 0, 7, 7, 7, 0, 0, 0, 0, 0, 0, 7, 7, 7, 8, 8, 8, 0, 0, 9, 0, 0, 0, 9, 9, 0, 0, 0, 0, 0, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 0, 0, 0, 0, 0, 0, 0], 'bboxes': [[881, 78, 1123, 158], [492, 117, 809, 246], [85, 137, 236, 184], [291, 135, 398, 181], [624, 255, 776, 316], [83, 280, 243, 327], [301, 280, 375, 327], [426, 310, 589, 391], [620, 316, 762, 391], [780, 315, 892, 391], [912, 314, 1043, 391], [1053, 316, 1141, 391], [985, 385, 1210, 469], [84, 434, 213, 482], [263, 434, 376, 481], [381, 434, 555, 481], [591, 395, 780, 469], [821, 391, 964, 469], [83, 466, 173, 501], [184, 465, 267, 501], [790, 475, 1015, 531], [696, 533, 1123, 612], [317, 614, 443, 686], [88, 652, 285, 690], [86, 688, 173, 727], [185, 688, 269, 727], [272, 688, 346, 727], [510, 760, 572, 837], [609, 760, 672, 837], [689, 760, 866, 837], [91, 797, 212, 838], [260, 797, 355, 838], [371, 797, 463, 838], [85, 831, 177, 875], [1

In [ ]:
label_list = ["O","B-key","B-information","B-name","B-rank","B-birth_date","B-nationality","B-death_date","B-funeral_date","B-grave_location","B-grave_id","B-information_source","B-death_book"]
id2label = {id:label for id,label in enumerate(label_list)}

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import torch

def normalize_bbox(bbox, width, height):
  # if any(coord < 0 for coord in bbox):
  #   raise ValueError(f"Invalid bbox coords: {bbox}")
  return [
      max(0, min(1000,int(1000 * (bbox[0] / width)))),
      max(0, min(1000,int(1000 * (bbox[1] / height)))),
      max(0, min(1000,int(1000 * (bbox[2] / width)))),
      max(0, min(1000,int(1000 * (bbox[3] / height)))),
  ]

class CustomDataset(Dataset):
  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    # get item
    example = self.dataset[idx]
    # image = Image.open(example["image_name"])
    words = example["words"]
    boxes = example["bboxes"]
    ner_tags = example["ner_tags"]

    # prepare for the model
    # width, height = image.size
    width, height = (1240, 1744)
    bbox = []
    labels = []
    for word, box, label in zip(words, boxes, ner_tags):
        box = normalize_bbox(box, width, height)
        n_word_tokens = len(tokenizer.tokenize(word))
        bbox.extend([box] * n_word_tokens)
        labels.extend([label] + ([-100] * (n_word_tokens - 1)))
        # labels.extend([label] * n_word_tokens)

    cls_box = sep_box = [0, 0, 0, 0]
    bbox = [cls_box] + bbox + [sep_box]
    labels = [-100] + labels + [-100]

    encoding = self.tokenizer(" ".join(words), truncation=True, max_length=512)
    sequence_length = len(encoding.input_ids)
    # truncate boxes and labels based on length of input ids
    labels = labels[:sequence_length]
    bbox = bbox[:sequence_length]

    encoding["bbox"] = bbox
    encoding["labels"] = labels

    return encoding

In [ ]:
#  %rm -rf lilt_robeczech_lr2e-05_bs4_train5/

In [ ]:
from transformers import AutoTokenizer

model_name, base_model, os.environ["WANDB_PROJECT"] = "nielsr/lilt-xlm-roberta-base","lilt_xlmroberta", "train_dataset_size_lilt"
# print(os.environ["WANDB_PROJECT"])
# os.environ["WANDB_PROJECT"] = "lilt_robeczech_train_size"

# model_name, base_model, os.environ["WANDB_PROJECT"] = "xkaska02/lilt-robeczech-base", "lilt_robeczech", "lilt_robeczech_train_size"
# model_name, base_model = "SCUT-DLVCLab/lilt-infoxlm-base", "lilt_infoxlm"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
train_dataset = CustomDataset(dataset["train"], tokenizer)
eval_dataset = CustomDataset(dataset["validation"], tokenizer)
test_dataset = CustomDataset(dataset["test"], tokenizer)

In [ ]:
#print(test_dataset[0])

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(features):
  """
  Collate function to prepare data for the model.

  This function pads sequences, handles labels and bounding boxes,
  and converts data to PyTorch tensors.

  Args:
      features (list): List of data samples.

  Returns:
      dict: Batched data ready for the model.
  """
  # Extract boxes, labels, and input_ids from features
  boxes = [feature["bbox"] for feature in features]
  labels = [feature["labels"] for feature in features]
  input_ids = [feature["input_ids"] for feature in features]

  # Pad sequences using the tokenizer
  # Use return_tensors='pt' to directly get PyTorch tensors
  batch = tokenizer.pad(
      {"input_ids": input_ids},
      padding="max_length",
      max_length=512,
      return_tensors="pt"
  )

  # Get the sequence length from the padded input_ids
  sequence_length = batch["input_ids"].shape[1]

  # Pad labels and boxes to match the sequence length
  # Ensure padding value is -100 for labels and [0, 0, 0, 0] for boxes
  batch["labels"] = torch.tensor([
      labels_example + [-100] * (sequence_length - len(labels_example))
      for labels_example in labels
  ])
  batch["bbox"] = torch.tensor([
      boxes_example + [[0, 0, 0, 0]] * (sequence_length - len(boxes_example))
      for boxes_example in boxes
  ])

  return batch

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
eval_dataloader = DataLoader(eval_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

In [ ]:
# batch = next(iter(train_dataloader))

In [ ]:
# for k,v in batch.items():
#   print(k, v.shape)

In [ ]:
def generate_model_name(base="lilt_robeczech", lr=5e-5, batch_size=8, train_size=30, extra=""):
  if extra != "":
    return f"{base}_lr{lr}_bs{batch_size}_train{train_size}_{extra}"
  else:
    return f"{base}_lr{lr}_bs{batch_size}_train{train_size}"

In [ ]:
# for example in eval_dataset:
#     assert len(example["input_ids"]) == len(example["labels"]), f"Mismatch in length: {len(example['input_ids'])} != {len(example['labels'])}"

In [ ]:
# for id, box, label in zip(batch["input_ids"][0], batch["bbox"][0], batch["labels"][0]):
#   if label.item() != -100:
#     print(tokenizer.decode([id]), box, id2label[label.item()])
#   else:
#     print(tokenizer.decode([id]), box, label.item())

In [ ]:
from transformers import LiltForTokenClassification

model = LiltForTokenClassification.from_pretrained(model_name, id2label=id2label)


Some weights of LiltForTokenClassification2layer were not initialized from the model checkpoint at nielsr/lilt-xlm-roberta-base and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate
metric = evaluate.load("seqeval")

In [ ]:
import numpy as np
from seqeval.metrics import classification_report

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [ ]:
lr = 2e-5
batch_size = 4
# base_model = "lilt_robeczech"
output_name = generate_model_name(base=base_model, lr=lr, batch_size=batch_size, train_size=len(dataset["train"]))

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(num_train_epochs=30,
                                  learning_rate=1e-5,
                                  eval_strategy="epoch",
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  metric_for_best_model="eval_loss",
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  run_name=output_name,
                                  report_to="wandb"
                                  # gradient_accumulation_steps=2
                                  )

In [ ]:
training_args.output_dir = output_name

In [ ]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.0001)

In [ ]:
from transformers.data.data_collator import default_data_collator

class CustomTrainer(Trainer):
  def get_train_dataloader(self):
    return train_dataloader

  def get_eval_dataloader(self, eval_dataset = None):
    return eval_dataloader

# Initialize our Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

<ipython-input-387-2dd4568404d2>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [ ]:
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.848304,0.487482,0.479479,0.483447,0.772708


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
trainer.push_to_hub()

In [ ]:
wandb.finish()